# Dependency Parsing

## Linguistic Structure
There are two views on linguistic structure.
1. Phrase structure organizes words into nested constituents.
    * So-called context-free grammars
    * Working up the Chomsky hierarchy
2. Dependency structure shows which words depend on which other words.
    * A key parsing decision is how we attach various constituents
    
We will focus on the second view on linguistic structure. 

## The Rise of Annotated Data
Treebank is a parsed text corpus that annotates syntactic or semantic sentence structure. 
![tree-bank](https://upload.wikimedia.org/wikipedia/commons/7/7d/The_house_at_the_end_of_the_street.jpg)

Building a tree bank seems a lot slower and less userful than coming up with an universal grammar rule. But a treebank gives us many times.
* Reusability of labor
* Broad coverage, not just few intuitions
* Frequencies and distributional information
* A way to evaluate systems

## Dependency Syntax
Dependency syntax postulates that syntactic structure consists of relations between lexical items, normally binary asymmetric relations ("arrows") called dependencies. The arrows are commonly **typed** with the name of grammatical relations (e.g. subject, prepositional object, apposition, etc...)

### Dependency Grammar History
The idea of dependency structure goes back a long way
* Panini's grammar in 5th century BCE
* Basic approach of 1st millenium Arabic grammarians
  
Constituency and context free grammars are new-fangled inventions.
* 20th century invention by R.S. Wells
    
Modern dependency work is often linked to work of L. Tesniere (1959)
* Dominant approach in the *East*

### Dependency Conditioning Preferences
What are the sources of information for dependency parsing?
1. Bilexical affinities
2. Dependency distance
3. Intervening material
4. Valency of heads

A sentence is parsed by choosing for each word what other word (including **ROOT**) is it a dependent of. Usually some constraints are applied.
* Only one word is a dependent of ROOT
* Dependency cannot be cyclic

This means dependency is a tree. The final issue is whether arrows can cross (non-projective) or not.

![dependency-tree](https://upload.wikimedia.org/wikipedia/commons/6/61/Latex-dependency-parse-example-with-tikz-dependency.png)

### Methods of Dependency Parsing
1. Dynamic programming - Eisner (1996) gives a clever algorithm with complexity O(n^3), b producing a parse items with heads at the ends rather than in the middle.
2. Graph algorithms - Create a minimum spanning tree for a sentence
3. Constraint satisfaction - Edges are eliminated that don't satisfy hard constraints
4. Transition-based parsing or deterministic dependency parsing - Greedy choice of attachments guided by good machine learning classifiers 

## Arc-standard Transition-based Parser
This is the primary parser architecture we will be using throughout CS224n. This is a simple form of greedy discriminative dependency parser. The parser does a sequence of bottom up actions.

The parser has
* A stack written with top to the right, which starts with ROOT symbol 
* A buffer, written with top to the left, which starts with the input sentence
* A set of dependency arcs `A` which starts off empty
* A set of actions
    * Shift
    * Left-arc
    * Right-arc

Here's an example.
```python
input_sentence = "I ate fish"
```

We start off with root in a stack and the tokens of the sentence in a buffer, and an empty set of arcs. 
```python
arcs = dict()
stack = ['ROOT']
buffer = ['I', 'ate', 'fish']
```

### Shift
We do a shift, which is taking the first word from the buffer (left to right ordering) and put it onto the stack.
```python
shift(stack, buffer)
print stack # => ['ROOT', 'I']
print buffer # => ['ate', 'fish'] 
```

We shift again!
```python
shift(stack, buffer)
print stack # => ['ROOT', 'I', 'ate']
print buffer # => ['fish'] 
```

### Left Arc 
Now we should be doing a left arc action. We remove the second top item from the stack and put it into the arc set because we can see that `'I'` is dependent of `'ate'`. 
```python
left_arc(stack, buffer, arcs)
print stack # => ['ROOT', 'ate']
print buffer # => ['fish']
print arcs # => { 'nsubj': ('ate', 'I') }
```

Shift again
```python
shift(stack, buffer)
print stack # => ['ROOT', 'ate', 'fish']
print buffer # => []
```

### Right Arc
At this point the buffer is empty. We do the final two right arc(s), which is to say the top of stack should be made a dependent of the thing that's second top item from the stack.
```python
right_arc(stack, buffer, arcs)
print stack # => ['ROOT', 'ate'] 
print buffer # => []
print arcs # => { 'nsubj': ('ate', 'I') , 'obj': ('ate', 'fish') }

right_arc(stack, buffer, arcs)
print stack # => ['ROOT']
print buffer # => []
print arcs # => { 'nsubj': ('ate', 'I') , 'obj': ('ate', 'fish'), 'root': ('ROOT', 'ate') }
```

### Action Decision?
Done! But how do we decide which action to take during parsing? We need to train a classifier to do that. More on this in next lecture.